In [1]:
!pip install inflection feature_engine category_encoders optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 276 kB 4.2 MB/s 
     |████████████████████████████████| 69 kB 6.9 MB/s 
     |████████████████████████████████| 308 kB 53.0 MB/s 
     |████████████████████████████████| 9.8 MB 22.9 MB/s 
     |████████████████████████████████| 209 kB 38.2 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 78 kB 3.7 MB/s 
     |████████████████████████████████| 146 kB 36.5 MB/s 
     |████████████████████████████████| 49 kB 3.3 MB/s 
     |████████████████████████████████| 112 kB 44.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=e60291709fa2c706fa2a34bcb5ddf037aa1022717281871af0e7c527f7839c75
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
  Attempting uninstall: statsmodels
  

In [ ]:
import inflection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import rcParams

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imblearn_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer
from feature_engine.creation import CyclicalFeatures
from feature_engine.discretisation import ArbitraryDiscretiser
from feature_engine.encoding import CountFrequencyEncoder, OneHotEncoder, RareLabelEncoder
from category_encoders.target_encoder import TargetEncoder

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_validate
from tqdm import tqdm

import optuna
import joblib
from optuna.samplers import TPESampler

def load_dataset(path):
    df = pd.read_csv(path)
    df.columns = [inflection.underscore(var) for var in list(df.columns)]    
    return df

def data_preparation(df):
    df = df.copy()
    
    # drop duplicated data
    df = df.drop_duplicates()
    
    # parsing to string
    variables = ['operating_systems', 'browser', 'region', 'traffic_type', 'weekend']
    df[variables] = df[variables].astype(str)
    
    # parsing month to integer
    df['month'] = df['month'].map({
        #'Jan': 1, 
        'Feb': 2, 'Mar': 3, 
        #'Apr': 4, 
        'May': 5,
        'June': 6, 'Jul': 7, 'Aug': 8, 
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    })
    
    # parsing revenue to integer
    df['revenue'] = df['revenue'].astype(int)
    
    return df

def split_dataset(df, target='revenue', test_size=0.2, seed=0):
    return train_test_split(
        df,
        test_size=test_size,
        random_state=seed,
        stratify=df[target]
    )

def random_forest_hyperspace(trial):
    return {
      "n_estimators": trial.suggest_int("n_estimators", 10, 300),
      "max_depth": trial.suggest_int("max_depth", 3, 20),
      "criterion": trial.suggest_categorical("criterion", ['gini','entropy']),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
      "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
      "class_weight": trial.suggest_categorical("class_weight", ['balanced','balanced_subsample']),
      "random_state": 0,
    }

def extra_trees_hyperspace(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 10, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "criterion": trial.suggest_categorical("criterion", ['gini','entropy']),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
        "class_weight": trial.suggest_categorical("class_weight", ['balanced','balanced_subsample']),
        "random_state": 0,
    }

def ada_boost_hyperspace(trial):
    return {
      "n_estimators": trial.suggest_int("n_estimators", 10, 300),
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.01, log=True),
      "algorithm": trial.suggest_categorical("algorithm", ['SAMME', 'SAMME.R']),
      "random_state": 0,
    }

def decision_tree_hyperspace(trial):
    return {
      "max_depth": trial.suggest_int("max_depth", 3, 20),
      "criterion": trial.suggest_categorical("criterion", ['gini','entropy']),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
      "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
      "class_weight": "balanced",
      "random_state": 0,
    }

def xgboost_hyperspace(trial):
    return {
      "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01, log=True),
      "n_estimators": trial.suggest_int("n_estimators", 10, 300),
      "max_depth": trial.suggest_int("max_depth", 3, 20),
      "min_child_weight": trial.suggest_int("min_child_weight", 2, 50),
      "subsample": trial.suggest_float("subsample", 0.5, 0.8),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8),
      "reg_alpha": trial.suggest_float("reg_alpha", 0, 0.2),
      "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-10, 100),
      "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1, 3),
      "random_state": 0,
      "objective": 'binary:logistic',
      "eval_metric": 'auc',
    }

def cross_validation(X, y, model):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    cv_results = cross_validate(
        model, 
        X, 
        y,
        cv=cv,
        scoring=["precision", "recall", "f1", "roc_auc"],
        return_estimator=True, 
        return_train_score=True,
    )
    display(
      {
        "features": cv_results['estimator'][0].named_steps['rfe'].get_feature_names_out(),

        "precision_train": f"{cv_results['train_precision'].mean():.3f}  ± {cv_results['train_precision'].std():.3f}",
        "precision_test": f"{cv_results['test_precision'].mean():.3f}  ± {cv_results['test_precision'].std():.3f}",

        "recall_train": f"{cv_results['train_recall'].mean():.3f}  ± {cv_results['train_recall'].std():.3f}",
        "recall_test": f"{cv_results['test_recall'].mean():.3f}  ± {cv_results['test_recall'].std():.3f}",

        "f1_train": f"{cv_results['train_f1'].mean():.3f}  ± {cv_results['train_f1'].std():.3f}",
        "f1_test": f"{cv_results['test_f1'].mean():.3f}  ± {cv_results['test_f1'].std():.3f}",

        "auc_train": f"{cv_results['train_roc_auc'].mean():.3f}  ± {cv_results['train_roc_auc'].std():.3f}",
        "auc_test": f"{cv_results['test_roc_auc'].mean():.3f}  ± {cv_results['test_roc_auc'].std():.3f}"
     }  
    )
    return round(cv_results['test_f1'].mean(), 3)

def objective(trial, X, y, hyperspace, model_obj):
    params = hyperspace(trial)
    n_features_to_select = trial.suggest_int("n_features_to_select", 1, X.shape[1])
    model = imblearn_pipeline(
        CyclicalFeatures(variables=['month'],drop_original=True),
        OneHotEncoder(variables=['visitor_type', 'weekend']),
        RFE(model_obj(random_state=0), n_features_to_select=n_features_to_select, step=1),
        SMOTE(random_state=0),
        model_obj(**params)
    )
    return cross_validation(X, y, model)

#if __name__ == "__main__":
def main():
    # data preparation
    df = load_dataset('./online_shoppers_intention.csv')
    df = data_preparation(df)
    df = df.drop(['operating_systems', 'browser', 'region', 'traffic_type'], axis=1)

    # split dataset into train and test
    full_train, test = split_dataset(df, target='revenue', test_size=0.2)

    # only full_train that use on the optuna study
    target = 'revenue'
    X, y = full_train.drop(target, axis=1), full_train[target]

    # specify the model and initialize the optuna study
    hyperspace_grid = [
      #decision_tree_hyperspace, 
      #ada_boost_hyperspace,
      random_forest_hyperspace,
      #xgboost_hyperspace,
      #extra_trees_hyperspace

    ]
    model_obj_grid = [
      #DecisionTreeClassifier, 
      #AdaBoostClassifier,
      RandomForestClassifier,
      #XGBClassifier,
      #ExtraTreesClassifier
    ]
    
    classifier_name = [
      #"DecisionTreeClassifier",
      #"AdaBoostClassifier",
      "RandomForestClassifier",
      #"XGBClassifier",
      #"ExtraTreesClassifier",
    ]
    
    for i, (hyperspace, model_obj) in enumerate(zip(hyperspace_grid, model_obj_grid)):
        func = lambda trial: objective(trial, X, y, hyperspace, model_obj)
        sampler = TPESampler(seed=0)
            
        study = optuna.create_study(
            direction='maximize', 
            sampler=sampler
        )
        study.optimize(func, n_trials=300)
        
        fig = optuna.visualization.plot_optimization_history(study)
        fig.show()
        
        saved_file = classifier_name[i]
        with open(f"{saved_file}_study.pkl", 'wb') as f_out:
            joblib.dump(study, f_out)
main()

[I 2022-06-17 07:39:42,506] A new study created in memory with name: no-name-5042b11b-e5f5-4adc-bfb1-d1d49153cfb4


{'auc_test': '0.930  ± 0.004',
 'auc_train': '0.948  ± 0.001',
 'f1_test': '0.684  ± 0.011',
 'f1_train': '0.705  ± 0.004',
 'features': array(['administrative', 'administrative_duration', 'informational',
        'informational_duration', 'product_related',
        'product_related_duration', 'bounce_rates', 'exit_rates',
        'page_values', 'month_sin', 'month_cos',
        'visitor_type_New_Visitor', 'weekend_False'], dtype=object),
 'precision_test': '0.619  ± 0.020',
 'precision_train': '0.640  ± 0.006',
 'recall_test': '0.766  ± 0.012',
 'recall_train': '0.784  ± 0.003'}

[I 2022-06-17 07:40:37,602] Trial 0 finished with value: 0.684 and parameters: {'n_estimators': 169, 'max_depth': 15, 'criterion': 'gini', 'min_samples_split': 22, 'min_samples_leaf': 33, 'class_weight': 'balanced_subsample', 'n_features_to_select': 13}. Best is trial 0 with value: 0.684.


{'auc_test': '0.931  ± 0.005',
 'auc_train': '0.964  ± 0.001',
 'f1_test': '0.690  ± 0.007',
 'f1_train': '0.742  ± 0.003',
 'features': array(['administrative', 'administrative_duration', 'informational',
        'informational_duration', 'product_related',
        'product_related_duration', 'bounce_rates', 'exit_rates',
        'page_values', 'month_sin', 'month_cos'], dtype=object),
 'precision_test': '0.639  ± 0.018',
 'precision_train': '0.693  ± 0.002',
 'recall_test': '0.751  ± 0.017',
 'recall_train': '0.799  ± 0.006'}

[I 2022-06-17 07:41:29,204] Trial 1 finished with value: 0.69 and parameters: {'n_estimators': 121, 'max_depth': 17, 'criterion': 'entropy', 'min_samples_split': 47, 'min_samples_leaf': 4, 'class_weight': 'balanced', 'n_features_to_select': 11}. Best is trial 1 with value: 0.69.


...

[I 2022-06-17 08:20:21,961] Trial 42 finished with value: 0.692 and parameters: {'n_estimators': 111, 'max_depth': 19, 'criterion': 'entropy', 'min_samples_split': 47, 'min_samples_leaf': 7, 'class_weight': 'balanced', 'n_features_to_select': 10}. Best is trial 10 with value: 0.693.


{'auc_test': '0.929  ± 0.006',
 'auc_train': '0.964  ± 0.001',
 'f1_test': '0.682  ± 0.009',
 'f1_train': '0.733  ± 0.002',
 'features': array(['administrative', 'administrative_duration',
        'informational_duration', 'product_related',
        'product_related_duration', 'bounce_rates', 'exit_rates',
        'page_values', 'month_sin'], dtype=object),
 'precision_test': '0.608  ± 0.018',
 'precision_train': '0.663  ± 0.006',
 'recall_test': '0.778  ± 0.016',
 'recall_train': '0.819  ± 0.005'}

[I 2022-06-17 08:21:28,938] Trial 43 finished with value: 0.682 and parameters: {'n_estimators': 138, 'max_depth': 17, 'criterion': 'entropy', 'min_samples_split': 43, 'min_samples_leaf': 7, 'class_weight': 'balanced', 'n_features_to_select': 9}. Best is trial 10 with value: 0.693.


{'auc_test': '0.931  ± 0.005',
 'auc_train': '0.958  ± 0.001',
 'f1_test': '0.691  ± 0.007',
 'f1_train': '0.728  ± 0.002',
 'features': array(['administrative', 'administrative_duration',
        'informational_duration', 'product_related',
        'product_related_duration', 'bounce_rates', 'exit_rates',
        'page_values', 'month_sin', 'month_cos'], dtype=object),
 'precision_test': '0.633  ± 0.012',
 'precision_train': '0.674  ± 0.005',
 'recall_test': '0.761  ± 0.017',
 'recall_train': '0.793  ± 0.007'}

[I 2022-06-17 08:22:25,458] Trial 44 finished with value: 0.691 and parameters: {'n_estimators': 119, 'max_depth': 19, 'criterion': 'entropy', 'min_samples_split': 47, 'min_samples_leaf': 14, 'class_weight': 'balanced', 'n_features_to_select': 10}. Best is trial 10 with value: 0.693.


In [ ]:
def best_trial_result(clf_name='DecisionTreeClassifier'):
    with open(f"{clf_name}_study.pkl", 'rb') as f_in:
        study = joblib.load(f_in)
    
    print("Best trial until now:")
    print(f"Classifier: {clf_name}")
    print(" Value (f1_test): ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
    print()
    return study

def main():
    classifier_name = [
      #'DecisionTreeClassifier',
      "RandomForestClassifier",
      #"ExtraTreesClassifier",
      #"AdaBoostClassifier",
      #"XGBClassifier",
    ]

    for name in classifier_name:
        display(best_trial_result(clf_name=name))

main()